# Model testing

This notebook provides a set of tests to run against `treat_sim`.  These tests are either pass or fail and no interpretation is needed. A summary of test results is provided at the end of the notebook.

> This notebook is a work in progress and more tests will be added in due course.

We have broken the testing of `treat_sim` into the following sections

* **Run tests:**.  These provide tests of the overall setup of the model including the ability to do repeatable single and multiple replications works as expected.
* **Extreme value tests:** The model is configured with extreme values for its parameters. For example, limiting arrivals, blocking routes, infinite capacity.
* **Deterministic runs:** Test if the model output behaves as expected if activites are converted to a deterministic value. 

## 1. Model Code Imports

In [1]:
from treat_sim.model import (
    Scenario, 
    TreatmentCentreModel,
    single_run, 
    multiple_replications
)

In [2]:
## these tests are applied to treat_sim version =
import treat_sim
treat_sim.__version__

'2.0.0'

## 2. Imports

In [3]:
import numpy as np
import pandas as pd
import statistics
import pytest
import ipytest

# types library is used to overwrite methods from treat_sim
import types

# sim_tools fixed dist is used for deterministic runs
from sim_tools.distributions import FixedDistribution

ipytest.autoconfig()

## 3. Tests

### 3.1 Model run test

Here we test that various modes of running the model work correctly.  These include

* single run mode
* multiple replications mode
* repeatable results using random number sets.
* results collection period



In [4]:
def test_single_run_results_length_and_type():
    '''
    Test a single_run of the model.
    
    The single_run function should return a pandas.DataFrame
    containing 16 columns and a single row.
    
     0   00_arrivals                    
     1   01a_triage_wait                 
     2   01b_triage_util               
     3   02a_registration_wait         
     4   02b_registration_util        
     5   03a_examination_wait          
     6   03b_examination_util          
     7   04a_treatment_wait(non_trauma)  
     8   04b_treatment_util(non_trauma)  
     9   05_total_time(non-trauma)       
     10  06a_trauma_wait               
     11  06b_trauma_util                 
     12  07a_treatment_wait(trauma)      
     13  07b_treatment_util(trauma)      
     14  08_total_time(trauma)           
     15  09_throughput                   

    Expected result: 
    ---------------
        len(run_results) == 16 and isinstance(run_results, pd.Dataframe)

    Returns:
    -------
    bool: does the model pass the test.
    '''
    EXPECTED_LENGTH = 16

    # a default experiment
    default_experiment_params = Scenario()

    # run the model in single run model
    run_results = single_run(default_experiment_params, random_no_set=41)

    # test
    assert len(run_results.T) == EXPECTED_LENGTH and isinstance(run_results, pd.DataFrame)

In [5]:
@pytest.mark.parametrize('n_reps', [
                          (1),
                          (2),
                          (10),
                          (23)
])
def test_multiple_replications_results_length_and_type(n_reps):
    '''
    Test running the model in multiple replications mode
    
    The multiple function should return a pandas.DataFrame
    containing 16 columns and n_reps rows. Its shape is (n_reps, 16)
    
     0   00_arrivals                    
     1   01a_triage_wait                 
     2   01b_triage_util               
     3   02a_registration_wait         
     4   02b_registration_util        
     5   03a_examination_wait          
     6   03b_examination_util          
     7   04a_treatment_wait(non_trauma)  
     8   04b_treatment_util(non_trauma)  
     9   05_total_time(non-trauma)       
     10  06a_trauma_wait               
     11  06b_trauma_util                 
     12  07a_treatment_wait(trauma)      
     13  07b_treatment_util(trauma)      
     14  08_total_time(trauma)           
     15  09_throughput                   

    Expected result: 
    ---------------
        rep_results.shape == (n_reps, 16) and isinstance(rep_results, pd.DataFrame)

    Returns:
    -------
    bool: does the model pass the test.
    '''
    EXPECTED_N_COLUMNS = 16
    EXPECTED_SHAPE = (n_reps, EXPECTED_N_COLUMNS)

    # a default experiment
    default_experiment_params = Scenario()

    # run the model in multiple replications mode
    rep_results = multiple_replications(default_experiment_params, n_reps=n_reps)

    # test
    assert rep_results.shape == EXPECTED_SHAPE and isinstance(rep_results, pd.DataFrame)

In [6]:
@pytest.mark.parametrize('random_number_set', [
                          (0),
                          (1),
                          (2),
                          (101),
                          (42),
])
def test_random_number_set(random_number_set):
    '''
    Test the model produces repeatable results
    given the same set set of random seeds.
    
    Expected result: 
    ---------------
        difference between data frames is 0.0
    '''

    results = []

    for i in range(2):
        
        exp = Scenario()

        # run the model in single run model
        run_results = single_run(exp, random_no_set=random_number_set)
    
        results.append(run_results)
        
    # test
    assert (results[0] - results[1]).sum().sum() == 0.0

In [7]:
@pytest.mark.parametrize('rc_period', [
                          (10.0),
                          (1_000.0),
                          (25.0),
                          (500.0),
                          (143.0),
])
def test_run_length_control(rc_period):
    scenario = Scenario()
    
    # set random number set - this controls sampling for the run.
    scenario.set_random_no_set(42)

    # create an instance of the model
    model = TreatmentCentreModel(scenario)

    # run the model
    model.run(results_collection_period=rc_period)
        
    # run results
    assert model.env.now == rc_period

### 3.2. Extreme value tests

Here we manipulate the input parameters of the model to test that it behaves as expected. We run the following tests

* All arrivals are trauma
* All arrivals are non-trauma
* Infinite capacity for activities
* All non-trauma patients require treatment
* All non-trauma patients do not require treatment

> To do: *Zero arrivals of both types"" -> Requires modification of `Scenario` 

> To investigate: blocked queues.  Simpy requires you to set resource count >=1; so cannot be simply blocked.  One way around this an arrival at time 0 and set activity time of the key activity to $M$ wa very large number. This means that the 1st patient arrives before the "real" arrival process begins, takes the 1 resource available and blocks the queue.  Do we actually need to do this?


In [8]:
### NOTE: we are ignoring mean of empty array warnings from numpy using filterwarnings
### We will handle this in a future release of treat_sim.

@pytest.mark.filterwarnings("ignore")
@pytest.mark.parametrize('random_no_set', [
                          (42),
                          (1),
                          (754),
                          (9876534321),
                          (76546783986555), 
])
def test_all_trauma(random_no_set):
    '''
    Force all patients to use trauma pathway
    
    i.e. Set prob_trauma = 1.0
    
    Expected result:
    -------
    No patients use the non-trauma pathway this means that a number of KPIs
    are NaN (not a number). These include:
    '02a_registration_wait'
    '05_total_time(non-trauma)'
    
    Trauma pathway KPIs are valid numbers. We test total time in 
    system: '08_total_time(trauma)'
    
    '''
    
    # create a new scenario and set prob of trauma to 100%
    scenario = Scenario(prob_trauma=1.0)

    # run the model in single run model
    run_results = single_run(scenario, random_no_set=random_no_set)
        
    # run results
    assert (pd.isna(run_results['05_total_time(non-trauma)'].iloc[0]) 
            and pd.isna(run_results['02a_registration_wait'].iloc[0]) 
            and not pd.isna(run_results['08_total_time(trauma)'].iloc[0]))

In [9]:
### NOTE: we are ignoring mean of empty array warnings from numpy using filterwarnings
### We will handle this in a future release of treat_sim.

@pytest.mark.filterwarnings("ignore")
@pytest.mark.parametrize('random_no_set', [
                          (42),
                          (1),
                          (754),
                          (9876534321),
                          (76546783986555), 
])
def test_all_nontrauma(random_no_set):
    '''
    Force all patients to use the non-trauma pathway
    
    i.e. Set prob_trauma = 0.0
    
    Expected result:
    -------
    No patients use the trauma pathway this means that a number of KPIs
    if NaN (Not a Number) as no patient use the activities.
    
    '06a_trauma_wait'
    '07a_treatment_wait(trauma)'
    '08_total_time(trauma)'
    
    Non-trauma pathway KPIs are valid numbers. We test
    '02a_registration_wait'
    '05_total_time(non-trauma)'
    '''
    
    # create a new scenario and set prob of trauma to 100%
    scenario = Scenario(prob_trauma=0.0)

    # run the model in single run model
    run_results = single_run(scenario, random_no_set=random_no_set)
        
    # run results
    assert (pd.isna(run_results['06a_trauma_wait'].iloc[0]) 
            and pd.isna(run_results['07a_treatment_wait(trauma)'].iloc[0]) 
            and pd.isna(run_results['08_total_time(trauma)'].iloc[0]) 
            and not pd.isna(run_results['02a_registration_wait'].iloc[0])
            and not pd.isna(run_results['05_total_time(non-trauma)'].iloc[0]))

In [10]:
@pytest.mark.parametrize('random_no_set', [
                          (42),
                          (1),
                          (754),
                          (9876534321),
                          (76546783986555),
                          (9876888854637815463789)
])
def test_infinite_capacity(random_no_set):
    '''
    Remove all capacity constraints in the model 
    by setting all capacity to M a large number
    
    Expected result:
    -------
    No queuing. The following KPIs = 0.0
    
    01a_triage_wait
    02a_registration_wait
    03a_examination_wait
    04a_treatment_wait(non_trauma)
    06a_trauma_wait'
    07a_treatment_wait(trauma)
    '''
    
    M = 100_000_000
    
    # create a new scenario and set prob of trauma to 100%
    scenario = Scenario(n_triage=M, n_reg=M, n_exam=M, n_trauma=M, 
                        n_cubicles_1=M, n_cubicles_2=M)

    # run the model in single run model
    run_results = single_run(scenario, random_no_set=random_no_set)
    
    
    assert (run_results['01a_triage_wait'].iloc[0] == 0.0
            and run_results['02a_registration_wait'].iloc[0] == 0.0
            and run_results['03a_examination_wait'].iloc[0] == 0.0
            and run_results['04a_treatment_wait(non_trauma)'].iloc[0] == 0.0
            and run_results['06a_trauma_wait'].iloc[0] == 0.0
            and run_results['07a_treatment_wait(trauma)'].iloc[0] == 0.0)

In [11]:
### NOTE: we are ignoring mean of empty array warnings from numpy using filterwarnings
### We will handle this in a future release of treat_sim.

@pytest.mark.filterwarnings("ignore")
@pytest.mark.parametrize('random_no_set', [
                          (42),
                          (1),
                          (754),
                          (9876534321),
                          (76546783986555), 
])
def test_no_treatment_for_nontrauma(random_no_set):
    '''
    Force all non-trauma patients to be discharged without treatment
    
    i.e. non_trauma_treat_p = 0.0
    
    Expected result:
    -------
    No non-trauma patients queue or use treatment cubicles
    The following variable is NaN

    04a_treatment_wait(non_trauma)
    
    and the utilisation of the non-trauma cubicles is 0
    
    04b_treatment_util(non_trauma)

    '''
    # create a new scenario and set prob of trauma to 100%
    scenario = Scenario(non_trauma_treat_p=0.0)

    # run the model in single run model
    run_results = single_run(scenario, random_no_set=random_no_set)
        
    # run results
    assert (pd.isna(run_results['04a_treatment_wait(non_trauma)'].iloc[0]) 
            and run_results['04b_treatment_util(non_trauma)'].iloc[0] == 0.0)

In [12]:
### NOTE: we are ignoring mean of empty array warnings from numpy using filterwarnings
### We will handle this in a future release of treat_sim.

@pytest.mark.filterwarnings("ignore")
@pytest.mark.parametrize('random_no_set', [
                          (42),
                          (1),
                          (754),
                          (9876534321),
                          (76546783986555), 
])
def test_all_nontrauma_no_treatment(random_no_set):
    '''
    Force all patients to use the non-trauma pathway
    
    i.e. Set prob_trauma = 0.0 and non_trauma_treat_p = 0.0
    
    Expected result:
    -------
    No patients use the trauma pathway this means that a number of KPIs
    if NaN (Not a Number) as no patient use the activities.
    
    '06a_trauma_wait'
    '07a_treatment_wait(trauma)'
    '08_total_time(trauma)'
    
    Non-trauma pathway KPIs are valid numbers. We test
    '02a_registration_wait'
    '05_total_time(non-trauma)'
    
    No non-trauma patients queue or use treatment cubicles
    The following variable is NaN

    04a_treatment_wait(non_trauma)
    
    and the utilisation of the non-trauma cubicles is 0
    
    04b_treatment_util(non_trauma)
    
    '''
    
    # create a new scenario and set prob of trauma to 100%
    scenario = Scenario(prob_trauma=0.0,
                        non_trauma_treat_p=0.0)

    # run the model in single run model
    run_results = single_run(scenario, random_no_set=random_no_set)
        
    # run results
    assert (pd.isna(run_results['06a_trauma_wait'].iloc[0]) 
            and pd.isna(run_results['07a_treatment_wait(trauma)'].iloc[0]) 
            and pd.isna(run_results['08_total_time(trauma)'].iloc[0]) 
            and not pd.isna(run_results['02a_registration_wait'].iloc[0])
            and not pd.isna(run_results['05_total_time(non-trauma)'].iloc[0])
            and pd.isna(run_results['04a_treatment_wait(non_trauma)'].iloc[0]) 
            and run_results['04b_treatment_util(non_trauma)'].iloc[0] == 0.0)

### 3.3 Deterministic activities

We have simplifed the model to a deterministic run by replacing all activity distributions with a fixed static value.

> To do: This will involve modifying the Scenario class distributions. Do we allow arrivals to remain stochastic (rate and patient type?). 

#### Functions used to overwrite `Scenario` in deterministic tests.

In [13]:
def init_sampling_to_fixed(self):
    '''
    Create the distributions used by the model and initialise 
    the random seeds of each.  This modified function
    makes use of a fixed distribution where the fixed value
    is the mean of the original sampling distribution.
    
    We ignore arrivals as this is handled by init_nspp()
    
    '''       
    # create distributions - all are fixed apart from arrivals

    # Triage duration
    self.triage_dist = FixedDistribution(self.triage_mean)

    # Registration duration (non-trauma only)
    self.reg_dist = FixedDistribution(self.reg_mean)

    # Evaluation (non-trauma only)
    self.exam_dist = FixedDistribution(self.exam_mean)

    # Trauma/stablisation duration (trauma only)
    self.trauma_dist = FixedDistribution(self.trauma_mean)

    # Non-trauma treatment
    self.nt_treat_dist = FixedDistribution(self.non_trauma_treat_mean)

    # treatment of trauma patients
    self.treat_dist = FixedDistribution(self.trauma_treat_mean)

    # probability of non-trauma patient requiring treatment
    self.nt_p_treat_dist = FixedDistribution(self.non_trauma_treat_p)

    # probability of non-trauma versus trauma patient
    self.p_trauma_dist = FixedDistribution(self.prob_trauma)

    # init sampling for non-stationary poisson process
    # we leave this as stochastic.
    self.init_nspp()

In [14]:
@pytest.mark.filterwarnings("ignore")
@pytest.mark.parametrize('random_no_set', [
                          (42),
                          (1),
                          (754),
                          (9876534321),
                          (76546783986555), 
])
def test_deterministic_nontrauma_pathway(random_no_set):
    '''
    Test a single_run of the model when the non_trauma
    pathway is set to determinstic values with no capacity 
    constraints.
    
    Arrivals are left as stochastic, this should not affect
    result.

    Expected result: 
    ---------------
        total time in system = 37.3 allowing for floating point error.
    '''
    # create a new scenario and set prob of trauma to 100%
    M = 1000_000_000

    # create a new scenario and set prob of trauma to 100%
    scenario = Scenario(n_triage=M, n_reg=M, n_exam=M, n_trauma=M, 
                        n_cubicles_1=M, n_cubicles_2=M, non_trauma_treat_p=1.0, prob_trauma=0.0)

    # calculate the expected time in system = sum of deterministic activity times.
    expected_total_time = scenario.triage_mean \
        + scenario.reg_mean + scenario.exam_mean + scenario.non_trauma_treat_mean

    # overwrite the function
    scenario.init_sampling = types.MethodType(init_sampling_to_fixed, scenario)
    
    # run the determinstic pathway (random numbers should make no difference)
    run_results = single_run(scenario, random_no_set=random_no_set)

    # test
    assert pytest.approx(run_results['05_total_time(non-trauma)'].iloc[0]) == expected_total_time

In [15]:
@pytest.mark.filterwarnings("ignore")
@pytest.mark.parametrize('random_no_set', [
                          (42),
                          (1),
                          (754),
                          (9876534321),
                          (76546783986555), 
])
def test_deterministic_trauma_pathway(random_no_set):
    '''
    Test a single_run of the model when the trauma
    pathway is set to determinstic values with no capacity 
    constraints.
    
    Arrivals are left as stochastic, this should not affect
    result.

    Expected result: 
    ---------------
        total time in system = sum(triage+stablisation+treatment) 
        allowing for floating point error.
    '''
    # create a new scenario and set prob of trauma to 100%
    M = 1000_000_000

    # create a new scenario and set prob of trauma to 100%
    scenario = Scenario(n_triage=M, n_reg=M, n_exam=M, n_trauma=M, 
                        n_cubicles_1=M, n_cubicles_2=M, prob_trauma=1.0)
    
    # calculate the expected time in system = sum of deterministic activity times.
    expected_total_time = scenario.triage_mean \
        + scenario.trauma_mean + scenario.trauma_treat_mean
    
    # overwrite the function
    scenario.init_sampling = types.MethodType(init_sampling_to_fixed, scenario)
    
    # run the determinstic pathway (random numbers should make no difference)
    run_results = single_run(scenario, random_no_set=random_no_set)

    # test
    assert pytest.approx(run_results['08_total_time(trauma)'].iloc[0]) == expected_total_time

## 4. Run all automated tests

In [16]:
ipytest.run("-vv", "--no-header")

======================================= test session starts ========================================
collecting ... collected 51 items

t_1406c931e1644556bb6da91cdf501505.py::test_single_run_results_length_and_type PASSED        [  1%]
t_1406c931e1644556bb6da91cdf501505.py::test_multiple_replications_results_length_and_type[1] PASSED [  3%]
t_1406c931e1644556bb6da91cdf501505.py::test_multiple_replications_results_length_and_type[2] PASSED [  5%]
t_1406c931e1644556bb6da91cdf501505.py::test_multiple_replications_results_length_and_type[10] PASSED [  7%]
t_1406c931e1644556bb6da91cdf501505.py::test_multiple_replications_results_length_and_type[23] PASSED [  9%]
t_1406c931e1644556bb6da91cdf501505.py::test_random_number_set[0] PASSED                      [ 11%]
t_1406c931e1644556bb6da91cdf501505.py::test_random_number_set[1] PASSED                      [ 13%]
t_1406c931e1644556bb6da91cdf501505.py::test_random_number_set[2] PASSED                      [ 15%]
t_1406c931e1644556bb6da91cdf501505

<ExitCode.OK: 0>